In [3]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import seaborn as sns

## Contexto

Los datos fueron recolectados y dispuestos por “National Institute of Diabetes and Digestive and Kidney Diseases”. El conjunto de datos ha sido filtrado y se han seleccionado instancias para evaluar pacientes pertenecientes al grupo indígena Pima (subgrupo de nativos Americanos).

Se consideran las siguientes variable para evaluar si un paciente tiene diabetes o no:

* **embarazo**: Número de embarazos
* **glucosa**: Concentración de glucosa en una prueba de tolerancia oral
* **bp**: Presión sanguínea (mmHg)
* **piel**: Espesor del pliegue cutáneo del tríceps (mm)
* **insulina**: Insulina en la sangre
* **bmi**: Indice de masa muscular
* **linaje**: Linaje de diabetes familar
* **edad**: Años
* **label**: Etiqueta 1 si tiene diabetes 0 si no

### Importar datos y definir variables

In [4]:
r = "pima-indians-diabetes.csv"
col_names = ['embarazo', 'glucosa', 'bp', 'piel', 'insulina', 'bmi', 'linaje', 'edad', 'label']
datos = pd.read_csv(r,sep=",", header=None, names=col_names)
datos.head()

FileNotFoundError: [Errno 2] No such file or directory: 'pima-indians-diabetes.csv'

### Exploración de datos

In [ ]:
corr = datos.corr()
sns.heatmap(corr, xticklabels=corr.columns, 
         yticklabels=corr.columns)
plt.show()

In [ ]:
Data = pd.DataFrame(np.array([[0,500],[1,268]]),columns=["label","Conteo"])
sns.barplot(x="label", y="Conteo", data=Data)
plt.show()

In [ ]:
# Seleccionar variables independiente
X = datos.drop(["label"],axis=1)
nombres = X.columns

# Seleccionar variables dependientes
y = datos["label"]

# Particionar datos de prueba y entrenamiento
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

### Generación del árbol

In [ ]:
Dtree = DecisionTreeClassifier(criterion="gini", max_depth=4, splitter="random")
Dtree.fit(X_train, y_train)

with open("arbol_1.dot","w") as f:
    f = tree.export_graphviz(Dtree, out_file=f, filled=True, special_characters=True,
                            feature_names = nombres,class_names=['0','1'])

Predict = Dtree.predict(X_test)    
print(classification_report(y_test,Predict))

### Evaluación de desempeño

In [ ]:
print(confusion_matrix(y_test,Predict))
print(classification_report(y_test,Predict))

### Pronóstico con valores de prueba

In [ ]:
Predict = Dtree.predict(X_test)

Pronos=[]
Observ=[]

# Se crea un contador para saber el número de veces que una clase es pronosticada
for i in range(0,2):
    p = 0
    for j in Predict:
        if i == j:
            p += 1
    Pronos.append(p)
    
# Se crea un contador para saber el número de veces que una clase es observada
for i in range(0,2):
    o = 0
    for j in y_test:
        if i == j:
            o += 1
    Observ.append(o)

print("Diabetes pornósticada y observada:")
for i in range(len(Pronos)):
    print(" Diabetes %s : %s - %s" %(i,Pronos[i],Observ[i]))


### Importancia de variables

In [ ]:
b = Dtree.feature_importances_
a = []
for i in range(4):
    a.append(b[i])
impor = pd.DataFrame(list(zip(nombres,a)),columns=["Variable","Importancia de Gini"])

print("Importancia de las variables:")
impor.sort_values(by="Importancia de Gini",ascending=False).head()

In [ ]:
Pronos = np.asarray(Pronos)
Observ = np.asarray(Observ)
A = y_test==Predict
V_count = 0

for i in A:
    if i==True:
        V_count+=1
        
Prev = V_count/len(Predict) # prev...alencia
print("El %.2f"%(Prev*100) + "% de los vinos estuvieron bien clasificados")